In [1]:
import input_files
import input_combi
import helpers

In [2]:
global_configs = {
        'write_directory' : '/gpfs/home/gdb20/computations/mike/stgaps_oct',
        'xyz_directory' : '/gpfs/home/gdb20/code/mol_maker/data/mike/mike_molecules',
        'charge': 0,
}
#need:
#to be able to group these into directories by name; 
#make name part of directory path and not basename as a flag

#need for now: make one of these from a directory of xyz files
 
nics_molecules = {
             'cby': {
                'xyz_file' : 'cby.xyz',
                    },
             'cmy': {
                'xyz_file' : 'cmy.xyz',
                    },
             'omy': {
                'xyz_file' : 'omy.xyz',
                     },
             'oby': {
                 'xyz_file': 'oby.xyz',
                     },
             'ome': {
                 'xyz_file': 'ome.xyz',
                     },
            '!directories' : True,
}
no_nics_molecules = {
             'ncmy': {
                 'xyz_file': 'ncmy.xyz',
                     },
             'noby': {
                 'xyz_file': 'noby.xyz',
                     },
             'nody2': {
                 'xyz_file': 'nody2.xyz',
                     },
             'nome': {
                 'xyz_file':'nome.xyz',
                     },
             'nome2': {
                 'xyz_file':'nome2.xyz',
                     },
             'nomph': {
                 'xyz_file':'nomph.xyz',
                     },
             'nomy': {
                 'xyz_file':'nomy.xyz',   
                     },
            'nomy2': {
                'xyz_file' : 'nomy2.xyz',
                    },
            'npome': {
                'xyz_file' : 'npome.xyz',
                    },
             '!directories' : True, #create directories with the full names so far for each of these
}
big_molecules = {
             'big_core': {
                 'xyz_file': 'big_core.xyz',
                     },
             'tbucore' : {
                 'xyz_file': 'tbucore.xyz',
                     },    
}

#################### PROGRAM SETTINGS ##############################

crest_settings = {'crest' : {
                    'program' : 'CREST',
                    'functional' : 'gfn2',
                    'quick' : 'quick',
                    'reopt' : True,
                    'cluster' : True,
                    'spin_multiplicity' : 3
                    }
                 }

#ideally, what we would do here:
#run crest at singlet and triplet spin
#make combined ensemble
#rerank with CENSO level 1 at both states
#.......
#even though I know we need to do this, I really don't know how to set this up.
#completely willing, but this will require a lot of testing.

#it would give us a good ensemble calculation, in addition to our detailed exploration of
#the lowest lying conformers

#NEED:
#script which combines ensembles in response to two CREST outputs being done
    #ability to make job dependent on multiple coordinates
    #ability to change coordinate names?

#CENSO to not be so noisy and make gigabytes of trash
    #(could modify it to serialize outputs and delete CC files)
    #or could make my own, but what's the point of that?

#CENSO to be able to interpret results of BS calculations 
    #to do concurrent rankings from one BS SP calc of singlet and triplets
    #to make singlet_best and triplet_best in this case...

#beyond that, this works as given


ORCA_settings = {
            'orca' : {
                'program' : 'orca',
                'integration_grid' : 'DEFGRID3', #tortoise and hare
                'other_keywords' : ['VeryTightSCF'], #TODO: scf_convergence setting
                '!coords_from': 'crest', 
                '!coords_file': 'crest_best.xyz', #
                },                      #where the whole name up to and including this phrase is included
           }

#computations:
#would be nice to test if these calculations go without issue using DEFGRID2 in ORCA 6

################# SCF and spin settings ######################

spin_scf_settings = {
            'triplet' : {
                'spin_multiplicity' : 3,
                'uks' : True,
            },
            'singlet' : {
                'spin_multiplicity' : 3,
                'broken_symmetry' : True,
                'uks' : True,
            }
}


################# LEVEL OF THEORY SETTINGS ##########################

#FOR ORCA CONFIG:
#ADD SCF CONVERGENCE TOLERANCE
#ADD PRINTING VERBOSITY
#ADD MOINP OPTION


#NEED: orbitals_from option

#these could just have a basename that is only the stuff not already included in their group directories
#less redundant, maybe easier to manipulate
#but most of all, this is how every other CC program and driver I've ever seen does it

methods = {    'r2SCAN-3c_opt_freq' : {
                'functional' : 'r2scan-3c',
                'basis_set' : None,
                },
                'wB97X-3c_sp' : { #this one will include options for rendering orbitals,
                #and densities
                'functional' : 'wB97X-3c',
                'basis_set' : None,
                'natural_orbitals' : True,
                'blocks' : [{'plots': [
                                'dim1 100',
                                'dim2 100',
                                'dim3 100',
                                'Format Gaussian_Cube',
                                'SpinDens("spindensity");'
                                'ElDens("electrondensity");',
                                ]
                            }]
                '!coords_from' : 'r2SCAN-3c_opt_freq',
                '!orbitals_from' : 'r2SCAN-3c_opt_freq',
                },
               'orbital_populations' : { #need to look up the ORCA syntax for this
                'functional' : 'wB97X-3c',
                'basis_set' : None,
                'other_keywords' : ['Normalprint','noiter','MOREAD'],
                #%moinp "basename.uno" #IMPLEMENT THIS
                '!coords_from' :  'wB97X-3c_sp',
                '!orbitals_from' : 'wB97X-3c_sp',
                '!orbitals_file' : '$basename$.uno' #TODO: implement this, .gbw by default
               },
               'wB97X-3c_tddft' : {
               'functional' : 'wB97X-3c',
               'basis_set' : 'none',
               '!coords_from' : 'r2SCAN-3c_opt_freq',
               '!orbitals_from' : 'r2SCAN-3c_opt_freq',
               },
}
#some script for requesting cube files

################### SOLVENT SETTINGS #############################

solvents = { #TODO: implement solvent model syntax in ORCA
    #            'gas' : {
    #             'solvent' : None
    #             },
            'thf' : {
                'solvent' : 'thf'
                },
           }

In [3]:
gaussian_inputs = input_combi.iterate_inputs([molecules,solvents,gaussian_settings,functionals,basis_sets])
crest_inputs = input_combi.iterate_inputs([molecules,solvents,crest_settings])

{'h2o2_gas_gau_b3lyp_6-31+Gdp': {'xyz_file': 'h2o2.xyz',
  'charge': 0,
  'spin_multiplicity': 1,
  'solvent': None,
  'program': 'Gaussian',
  '__coords_from__': 'crest',
  '__coords_file__': 'crest_best.xyz',
  'functional': 'b3lyp',
  'basis': '6-31+G*',
  'job_basename': 'h2o2_gas_gau_b3lyp_6-31+Gdp'},
 'h2o2_gas_gau_b3lyp_6-311+Gdp': {'xyz_file': 'h2o2.xyz',
  'charge': 0,
  'spin_multiplicity': 1,
  'solvent': None,
  'program': 'Gaussian',
  '__coords_from__': 'crest',
  '__coords_file__': 'crest_best.xyz',
  'functional': 'b3lyp',
  'basis': '6-311+G*',
  'job_basename': 'h2o2_gas_gau_b3lyp_6-311+Gdp'},
 'h2o2_gas_gau_b3lyp_6-31++Gdp': {'xyz_file': 'h2o2.xyz',
  'charge': 0,
  'spin_multiplicity': 1,
  'solvent': None,
  'program': 'Gaussian',
  '__coords_from__': 'crest',
  '__coords_file__': 'crest_best.xyz',
  'functional': 'b3lyp',
  'basis': '6-31++G*',
  'job_basename': 'h2o2_gas_gau_b3lyp_6-31++Gdp'},
 'h2o2_gas_gau_b3lyp_6-311++Gdp': {'xyz_file': 'h2o2.xyz',
  'charge':